# Load local dataset

In [1]:
import importlib
import random
import argparse
import configparser
import numpy as np
import networkx as nx
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_sparse
from torch import Tensor
from torch.nn import Linear
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch.optim as optim

from torch_geometric.utils import negative_sampling, to_networkx
from typing import Union, Tuple
from torch_geometric.typing import OptPairTensor, Adj, OptTensor, Size
from torch_sparse import SparseTensor, matmul
from torch_geometric.nn.conv import MessagePassing

from ogb.linkproppred import PygLinkPropPredDataset, Evaluator


import networkx as nx
import seaborn as sns
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

import scipy
import math


from dataset_utils import node_feature_utils
from dataset_utils.node_feature_utils import *
import my_utils as utils

importlib.reload(utils)

<module 'my_utils' from '/li_zhengdao/github/GenerativeGNN/my_utils.py'>

In [2]:
# Load specific dataset:

import sys,os
sys.path.append(os.getcwd())


from PrepareDatasets import DATASETS
import my_utils
import dataset_utils


print(DATASETS.keys())
"""
    'REDDIT-BINARY': RedditBinary,
    'REDDIT-MULTI-5K': Reddit5K,
    'COLLAB': Collab,
    'IMDB-BINARY': IMDBBinary,
    'IMDB-MULTI': IMDBMulti,
    'NCI1': NCI1,
    'ENZYMES': Enzymes,
    'PROTEINS': Proteins,
    'DD': DD,
    "MUTAG": Mutag,
    'CSL': CSL
"""

data_names = ['MUTAG']
datasets_obj = {}
for k, v in DATASETS.items():
    if k not in data_names:
        continue
    
    print('loaded dataset, name:', k)
    dat = v(use_node_attrs=True)
    datasets_obj[k] = dat
    print(type(dat.dataset.get_data()))


dict_keys(['REDDIT-BINARY', 'REDDIT-MULTI-5K', 'COLLAB', 'IMDB-BINARY', 'IMDB-MULTI', 'NCI1', 'ENZYMES', 'PROTEINS', 'DD', 'MUTAG', 'CSL'])
loaded dataset, name: MUTAG
processed_dir:  gnn_comparison/DATA/MUTAG/processed
load dataset !
dataset len:  188
<class 'list'>


In [3]:
# transform to grakel format or just load shuffle index??


mutag_data = datasets_obj['MUTAG'].get_test_fold(1, batch_size=1, shuffle=True)



In [5]:
fold_subset = mutag_data.dataset.get_subset()



19


In [10]:
def get_each_folder(fold_id, batch_size=1):
    
    fold_test = datasets_obj['MUTAG'].get_test_fold(fold_id, batch_size=batch_size, shuffle=True)
    fold_train, fold_val = datasets_obj['MUTAG'].get_model_selection_fold(fold_id, inner_idx=None, batch_size=batch_size, shuffle=True)
    
    train_y = []
    
    G_train = []
    train_y = []
    
    G_test = []
    test_y = []
    
    for d in fold_train:
        d = d.dataset
        
        G_train.append((d.to_numpy_array(), d.x))
        train_y.append(d.y.item())

    for d in fold_test:
        d = d.dataset
        
        G_test.append((d.to_numpy_array(), d.x))
        test_y.append(d.y.item())
        
    return G_train, G_test, train_y, test_y
    # do not use val for kernel methods.
#     for d in fold.dataset.get_subset():

In [11]:
from __future__ import print_function
print(__doc__)

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from grakel.datasets import fetch_dataset
from grakel.kernels import ShortestPath

# Loads the MUTAG dataset
MUTAG = fetch_dataset("MUTAG", verbose=False)
G, y = MUTAG.data, MUTAG.target

# Splits the dataset into a training and a test set
res=[]
for i in range(1):
    G_train, G_test, y_train, y_test = get_each_folder(i)
    # G_train, G_test, y_train, y_test = train_test_split(G, y, test_size=0.1)
    print(len(G_train))
    print(G_train[1])
    # Uses the shortest path kernel to generate the kernel matrices
    gk = ShortestPath(normalize=True)
    K_train = gk.fit_transform(G_train)
    K_test = gk.transform(G_test)

    # Uses the SVM classifier to perform classification
    clf = SVC(kernel="precomputed")
    clf.fit(K_train, y_train)
    y_pred = clf.predict(K_test)

    # Computes and prints the classification accuracy
    acc = accuracy_score(y_test, y_pred)
    res.append(acc)
    print("Accuracy:", str(round(acc*100, 2)) + "%")
    
    
res = np.array(res)
print(f'Acc, mean: {np.mean(res)}, std: {np.std(res)}')


    'REDDIT-BINARY': RedditBinary,
    'REDDIT-MULTI-5K': Reddit5K,
    'COLLAB': Collab,
    'IMDB-BINARY': IMDBBinary,
    'IMDB-MULTI': IMDBMulti,
    'NCI1': NCI1,
    'ENZYMES': Enzymes,
    'PROTEINS': Proteins,
    'DD': DD,
    "MUTAG": Mutag,
    'CSL': CSL



AttributeError: 'Batch' object has no attribute 'dataset'

In [5]:
print('len:', len(G))
print(G[0])

len: 188
[{(15, 13), (10, 11), (5, 6), (9, 8), (2, 1), (14, 13), (8, 9), (15, 16), (1, 6), (14, 9), (17, 15), (1, 2), (11, 10), (13, 12), (12, 13), (16, 15), (3, 4), (10, 9), (4, 10), (3, 2), (5, 4), (9, 14), (10, 4), (4, 5), (13, 15), (9, 10), (7, 5), (2, 3), (8, 7), (12, 11), (11, 12), (6, 5), (15, 17), (13, 14), (6, 1), (5, 7), (4, 3), (7, 8)}, {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 1, 16: 2, 17: 2}, {(2, 1): 0, (1, 2): 0, (3, 2): 0, (2, 3): 0, (4, 3): 0, (3, 4): 0, (5, 4): 0, (4, 5): 0, (6, 5): 0, (5, 6): 0, (6, 1): 0, (1, 6): 0, (7, 5): 0, (5, 7): 0, (8, 7): 0, (7, 8): 0, (9, 8): 0, (8, 9): 0, (10, 9): 0, (9, 10): 0, (10, 4): 0, (4, 10): 0, (11, 10): 0, (10, 11): 0, (12, 11): 0, (11, 12): 0, (13, 12): 0, (12, 13): 0, (14, 13): 0, (13, 14): 0, (14, 9): 0, (9, 14): 0, (15, 13): 1, (13, 15): 1, (16, 15): 2, (15, 16): 2, (17, 15): 1, (15, 17): 1}]
